Exercises adapted/copied from https://asecuritysite.com/

In [1]:
# let p = 7
# What generator points g are valid?

def getG(p):
    primitive_roots = []
    non_primitive_roots = []
    for g in range(1, p):
        seen = set()
        for exp in range(1, p):
            Y = pow(g, exp, p)  # Calculate Y = g^exp mod p
            if Y in seen:
                non_primitive_roots.append(g)
                break
            seen.add(Y)
        
        if len(seen) == p - 1:
            primitive_roots.append(g)
    
    return primitive_roots, non_primitive_roots

p = 7
print("p:", p)
# print("Values for g^x mod p:")
(primitive_roots, non_primitive_roots) = getG(p)
print("Primitive roots:", primitive_roots)
print("Non-primitive roots:", non_primitive_roots)

p: 7
Primitive roots: [3, 5]
Non-primitive roots: [1, 2, 4, 6]


# Schnorr interactive zkp

In [112]:
import random

def zero_knowledge_proof():
    # Initialize constants
    w = 3  # Prover's private witness
    g = 5  # Generator
    q = 97  # Prime number

    # Prover generates "public key"
    h = pow(g, w, q)  # h = g^w mod q
    print(f"prime q: {q} ...generator g: {g} ...private witness: {w} ...public_h: {h}")

    # Prover generates "random r"
    r1 = random.randint(1, q-1)
    a = pow(g, r1, q)  # a = g^r mod q
    print(f"prover generates random r and sends a = g^r mod q: {a}")

    # Verifier generates "challenge"
    e = random.randint(1, q-1)
    print(f"verifier generates a challenge e: {e}")

    # Prover generates "response" for verifier
    z = (r1 + e * w)
    print(f"prover's response z: {z}")

    # Verifier does equality check
    lhs = pow(g, z, q)  # lhs = g^z mod q
    rhs = (a * pow(h, e, q)) % q  # rhs = a * h^e mod qa

    print(f"lhs: {lhs}")
    print(f"rhs: {rhs}")

    if lhs == rhs:
        print("Verification successful!")
    else:
        print("Verification failed!")

# Run the zero-knowledge proof
zero_knowledge_proof()


prime q: 97 ...generator g: 5 ...private witness: 3 ...public_h: 28
prover generates random r and sends a = g^r mod q: 75
verifier generates a challenge e: 68
prover's response z: 276
lhs: 47
rhs: 47
Verification successful!


# Schnorr NIZK

In [7]:
import random
import hashlib

def hash_message(message):
    return int(hashlib.sha256(message).hexdigest(), 16)

def non_interactive_zero_knowledge_proof():
    # Initialize constants
    w = 3  # Prover's private witness
    g = 5  # Generator
    q = 97  # Prime number

    # Prover generates "public key"
    h = pow(g, w, q)  # h = g^w mod q
    print(f"prime q: {q} ...generator g: {g} ...private witness: {w} ...public_h: {h}")

    # Prover generates "random r"
    r1 = random.randint(1, q-1)
    a = pow(g, r1, q)  # a = g^r mod q

    # Prover generates "challenge" using a hash function
    #     gap where q is small, error rate is large
    # hash into a group is hard
    e = hash_message((str(a) + str(g) + str(q) + str(h)).encode('utf-8')) % q
    print(f"prover generates a challenge e using hash: {e}")

    # Prover generates "response" for verifier
    z = (r1 + e * w)
    print(f"prover's response z: {z}")

    # Verifier does equality check
    lhs = pow(g, z, q)  # lhs = g^z mod q
    rhs = (a * pow(h, e, q)) % q  # rhs = a * h^e mod q

    print(f"lhs: {lhs}")
    print(f"rhs: {rhs}")

    if lhs == rhs:
        print("Verification successful!")
    else:
        print("Verification failed!")

# Run the non-interactive zero-knowledge proof
non_interactive_zero_knowledge_proof()


prime q: 97 ...generator g: 5 ...private witness: 3 ...public_h: 28
prover generates a challenge e using hash: 1
prover's response z: 76
lhs: 24
rhs: 24
Verification successful!


# zk-SNARK - Discrete log additive homomorphism

In [9]:
import sys
import random

p=101
g= 3

# implicit assumption that x + y < 100 (smaller than the group -1)
# done via range proof in private transaction
x = 3
y = 4

ans=7


E1= g**( (x+y) % (p-1)) % p
# why p - 1
# 

E2= (g**x * g**y) % p

E3 = g**(ans) % p

print("prime: ", p, " generator: ", g, " answer: ", ans, " x: ", x, " y: ", y)

print('g^x modp=',(g**x) % p)
print('g^y modp=',(g**y) % p)

print('E1=',E1)
print('E2=',E2)
print('E3=',E3)

if (E2==E3):
	print('Alice has proven she knows the sum is ',ans)
else:
	print('Alice has proven she does not know the sum is ',ans)

prime:  101  generator:  3  answer:  7  x:  3  y:  4
g^x modp= 27
g^y modp= 81
E1= 66
E2= 66
E3= 66
Alice has proven she knows the sum is  7


# zk-SNARK Discrete Log Polynomial Commitment

In [16]:
import sys
import random

n=101
g= 3

# Bob's secret x
x=5

# Alice's  a and b
a = 3
b = 4

# eqn = ax + b x^2

E1= pow(g,( a *x ) ,n)

E2= pow(g,(b*x*x),n)

E3 = (E1 * E2) % n
E4 = pow(g,(a*x + b*x*x) , n)


print('======Agreed parameters============')
print('P=',n,'\t(Prime number)')
print('G=',g,'\t(Generator)')
print('a=',a)
print('b=',b)
print('x=',x,'\t(Eqn= ax + bx^2)')

print('E3=',E3)
print('E4=',E4)

if (E3==E4):
	print('Alice has computed the result')
else:
	print('Alice has proven she does not know result')

======Agreed parameters============
P= 101 	(Prime number)
G= 3 	(Generator)
a= 3
b= 4
x= 5 	(Eqn= ax + bx^2)
E3= 39
E4= 39
Alice has computed the result
